<a href="https://colab.research.google.com/github/makanadani/vetbot/blob/main/vetbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
# Instala a ferramenta de IA Generativa do Gemini
!pip install -q google.genai

# Configura a API Key do Google Gemini
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini
from google import genai
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

In [100]:
# Instala framework de agentes do Google
!pip install -q google-adk

# Importa ferramentas necessárias para construção do agente
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings
import re # Para validar o CEP
warnings.filterwarnings("ignore")

In [101]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:

    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()

    # Cria uma nova sessão
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")

    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""

    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [102]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [103]:
#################################################
# --------- Agente 1: VetBot BUSCADOR --------- #
#################################################

def vetbot_buscador(sintoma, especie, idade, raca, porte, sexo, castrado, cep, data_de_hoje):
  buscador = Agent(
      name="vetbot_buscador",
      model="gemini-2.0-flash",
      description="Agente que busca informações veterinárias confiáveis no Google.",
      tools=[google_search],
      instruction="""
      Você é um assistente de pesquisa veterinária, treinado para buscar informações educativas sobre saúde e comportamento de cães e gatos no
      Google (google_search) relacionadas ao assunto abaixo, considerando a espécie, idade, raça, porte e sexo do animal. Busque por informações sobre
      o sintoma/comportamento do animal descrito, priorizando informações de fontes confiáveis e reconhecidas na medicina veterinária, como sites de
      universidades, associações veterinárias nacionais e internacionais (.edu, .gov, .org de renome) ou artigos com base científica.
      Tente focar em informações publicadas no último ano, sempre que possível. Busque também por hospitais ou clínicas veterinárias 24 horas próximos
      ao CEP mencionado, conferindo se estão na mesma cidade e excluindo os que não estejam.
      """
  )
  entrada_vetbot_buscador = f"""
  Por favor, pesquise sobre o seguinte:
  - Sintoma/Comportamento: {sintoma}
  - Espécie: {especie}
  - Idade: {idade}
  - Raça: {raca if raca else 'Sem raça definida'}
  - Porte: {porte}
  - Sexo: {sexo}
  - Castrado(a): {castrado}
  - CEP: {cep}
  - Data atual: {data_de_hoje}
  """
  informacoes_gerais = call_agent(buscador, entrada_vetbot_buscador)
  return informacoes_gerais

In [104]:
#################################################
# -------- Agente 2: VetBot FILTRADOR -------- #
#################################################

def vetbot_filtrador(sintoma, especie, idade, raca, porte, sexo, castrado, cep, informacoes_buscadas):
    filtrador = Agent(
        name="vetbot_filtrador",
        model="gemini-2.0-flash",
        description="Agente que filtra, valida e sintetiza informações veterinárias com base científica.",
        tools=[google_search],
        instruction="""
        Você é um assistente científico e veterinário, especializado em estudos de cães e gatos. Sua tarefa é analisar as informações encontradas sobre
        o sintoma/comportamento no animal cujos dados constam na entrada. Com base na lista de informações e fontes fornecidas, sintetize os principais
        pontos de forma clara e didática para um tutor. Priorize informações com aparente base científica ou clínica, descartando explicitamente ou
        tratando com extrema cautela informações de blogs pessoais, fóruns e sites sem fontes científicas claras ou que pareçam anedóticas. Foque no
        conhecimento embasado sobre as possíveis causas gerais, o que observar em casa, e os níveis de urgência relevantes para um animal da espécie,
        idade, raça, porte e sexo apresentados. Mantenha um tom informativo e educativo, removendo sugestões de diagnósticos e de tratamentos. Liste os
        hospitais e clínicas veterinários 24 horas recebidos.
        """
    )
    entrada_vetbot_filtrador = f"""
    Detalhes do Animal:
    - Espécie: {especie}
    - Idade: {idade}
    - Raça: {raca if raca else 'Sem raça definida'}
    - Porte: {porte}
    - Sexo: {sexo if sexo else 'Não informado'}
    - Castrado(a): {castrado}
    - CEP: {cep}
    - Sintoma/Comportamento principal: {sintoma}
    A seguir, estão as informações encontradas na pesquisa inicial.
    Por favor, analise-as, filtre o conteúdo confiável e sintetize os pontos principais relevantes para este animal: {informacoes_buscadas}
    """
    informacoes_filtradas = call_agent(filtrador, entrada_vetbot_filtrador)
    return informacoes_filtradas

In [105]:
##########################################
# ------ Agente 3: VetBot REVISOR ------ #
##########################################
def vetbot_revisor(nome_pet, sintoma, especie, idade, raca, porte, sexo, castrado, cep, resposta_gerada):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        description="Agente revisor final para garantir segurança, clareza e relevância veterinária.",
        instruction="""
        Você é um médico veterinário experiente, especializado em atendimento de cães e gatos. Sua função é revisar o texto gerado para o tutor do animal
        descrito. Como veterinário, você sabe a importância de não realizar prescrições, diagnósticos ou sugerir tratamentos sem uma consulta presencial e
        exames. Revise o texto fornecido (o "Rascunho") para:
        1. Exclua quaisquer informações que possam configurar prescrição, diagnóstico ou orientação sobre tratamentos específicos (nomes de remédios,
        doses, procedimentos, o que fazer, etc.).
        2. Mantenha apenas o que for relevante para um tutor entender o sintoma ou comportamento apresentado no contexto da espécie, idade, raça, porte e
        sexo do animal. Isso inclui informações sobre o comportamento normal da espécie, possíveis causas gerais, o que observar em casa e o que não fazer,
        desmistificando medidas não científicas.  Use uma linguagem clara, empática e acessível ao tutor do animal.
        3. Obrigatório: Liste o nível de urgência aparente denre a lista: Emergência, Consulta recomendada, Monitoramento e Comportamento comum.
        4. Obrigatório: Personalize a resposta utilizando o nome real do pet e levando em consideração seus dados nas informações fornecidas.
        5. Obrigatório: Reforce claramente em sua resposta final (preferencialmente no início e no fim) que você é apenas um assistente virtual e
        não substitui o atendimento médico veterinário especializado.
        6. Obrigatório: Liste os hospitais ou clínicas veterinários 24 horas recebidos.
        7. Quando necessário, oriente o tutor sobre os riscos de não buscar esta consulta, considerando a possível gravidade implícita do sintoma
        para um animal com as características apresentadas. Use as categorias de nível de atenção (Emergência, Consulta recomendada, etc.) de forma clara.
        """
    )
    entrada_vetbot_revisor = f"""
    Detalhes do Animal:
    - Nome do Pet: {nome_pet}
    - Espécie: {especie}
    - Idade: {idade}
    - Raça: {raca if raca else 'Sem raça definida'}
    - Porte: {porte}
    - Sexo: {sexo if sexo else 'Não informado'}
    - Castrado(a): {castrado}
    - CEP: {cep}
    - Sintoma/Comportamento principal: {sintoma}
    Rascunho da resposta a ser revisada:
    {resposta_gerada}
    """
    resposta_final = call_agent(revisor, entrada_vetbot_revisor)
    return resposta_final

In [106]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🐾 Iniciando o VetBot – O Assistente Virtual do Seu Pet! 🐾")

# Coleta do NOME DO PET
nome_pet = input("😺🐶 Qual é o NOME do seu pet? ").capitalize()

# Coleta de ESPÉCIE
especie = ""
especie_opcoes = {
    "1": "Cão",
    "2": "Gato"
}
while especie == "":
    print("\n🐾 Qual a ESPÉCIE do seu animal?")
    print(f"1. {especie_opcoes['1']} 🐕")
    print(f"2. {especie_opcoes['2']} 🐈")
    escolha_especie = input("Digite o NÚMERO correspondente: ")
    if escolha_especie in especie_opcoes:
        especie = especie_opcoes[escolha_especie].lower() # Armazena em minúsculas para consistência
    else:
        print("❌ Opção inválida. Por favor, escolha 1 para Cão ou 2 para Gato.")

print(f"Você selecionou: {especie_opcoes[escolha_especie].capitalize()}")

# Coleta de IDADE com opções categorizadas
idade = ""
idade_opcoes = {
    "1": "Filhote (até 6 meses)",
    "2": "Jovem (de 7 a 18 meses)",
    "3": "Adulto (de 19 meses a 7 anos)",
    "4": "Meia-idade (de 7 a 10 anos)",
    "5": "Idoso (acima de 10 anos)",
    "6": "Não sei / Outra"
}
while idade == "":
    print(f"\n🎂 Qual a FAIXA DE IDADE aproximada de {nome_pet}?")
    for key, value in idade_opcoes.items():
        print(f"{key}. {value}")
    escolha_idade = input("Digite o NÚMERO correspondente: ")
    if escolha_idade in idade_opcoes:
        idade = idade_opcoes[escolha_idade].lower()
    else:
        print("❌ Opção inválida. Por favor, escolha uma das opções.")
print(f"Você selecionou: {idade_opcoes[escolha_idade]}")

# Coleta de RAÇA
raca = input(f"\n🦴 Qual a RAÇA de {nome_pet}? (OPCIONAL. Ex: Labrador, Siamês) ").strip().capitalize()

# Coleta de PORTE (PARA CÃES)
porte = "não aplicável"
if especie == "cão":
    porte_opcoes = {
        "1": "Pequeno",
        "2": "Médio",
        "3": "Grande",
        "4": "Gigante",
        "5": "Não sei"
    }
    # Loop até escolher uma opção válida
    while porte == "não aplicável":
        print(f"\n📏 Qual o PORTE de {nome_pet} ({especie})?")
        for key, value in porte_opcoes.items():
            print(f"{key}. {value}")
        escolha_porte = input("Digite o NÚMERO correspondente: ")
        if escolha_porte in porte_opcoes:
            porte = porte_opcoes[escolha_porte].lower() if porte_opcoes[escolha_porte] != "Não sei" else "não informado"
        else:
            print("❌ Opção inválida. Por favor, escolha uma das opções.")

    if porte != "não informado":
         print(f"Você selecionou: {porte_opcoes[escolha_porte]}")
else:
    print(f"\n📏 Porte não se aplica para gatos.")

# Coleta de SEXO
sexo = ""
sexo_opcoes = {
    "1": "Macho",
    "2": "Fêmea",
    "3": "Não sei / Não informado"
}
while sexo == "":
    print(f"\n🤔 Qual o SEXO de {nome_pet} ({especie})?")
    for key, value in sexo_opcoes.items():
        print(f"{key}. {value}")
    escolha_sexo = input("Digite o NÚMERO correspondente: ")
    if escolha_sexo in sexo_opcoes:
        sexo = sexo_opcoes[escolha_sexo].lower() if sexo_opcoes[escolha_sexo] != "Não sei / Não informado" else "não informado"
    else:
        print("❌ Opção inválida. Por favor, escolha uma das opções.")

if sexo != "não informado":
    print(f"Você selecionou: {sexo_opcoes[escolha_sexo].capitalize()}") # Confirmação
else:
    print("Sexo: Não informado.")

# Coleta de CASTRADO
castrado = ""
castrado_opcoes = {
    "1": "Sim",
    "2": "Não",
    "3": "Não sei"
}
while castrado == "":
    print(f"\n❤️ {nome_pet} é CASTRADO(A)?")
    for key, value in castrado_opcoes.items():
        print(f"{key}. {value}")
    escolha_castrado = input("Digite o NÚMERO correspondente: ")
    if escolha_castrado in castrado_opcoes:
        castrado = castrado_opcoes[escolha_castrado].lower() if castrado_opcoes[escolha_castrado] != "Não sei" else "não informado"
    else:
        print("❌ Opção inválida. Por favor, escolha uma das opções.")

if sexo != "não informado":
    print(f"Você selecionou: {castrado_opcoes[escolha_castrado].capitalize()}")
else:
    print("Castrado: Não informado.")

# Coleta do CEP para buscar hospitais
cep = ""

# Loop para continuar pedindo até que um CEP válido ou vazio seja digitado
while True:
    cep_input = input(f"\n📍 Para sugerir hospitais/clínicas próximos, qual o CEP da sua região? (OPCIONAL) ").strip()

    if not cep_input: # Verifica se o usuário deixou em branco
        print("CEP não informado.")
        cep = ""
        break

    # Expressão regular para validar CEP no formato ddddd-ddd ou dddddddd
    cep_padrao = re.compile(r'^\d{5}-?\d{3}$')

    if cep_padrao.fullmatch(cep_input):
        # CEP válido encontrado
        cep = cep_input.replace('-', '') # Remove o hífen para armazenar só os 8 dígitos numéricos
        print(f"CEP informado: {cep_input}") # Confirma para o usuário o que ele digitou
        break
    else:
        # Formato inválido
        print("❌ Formato de CEP inválido. Por favor, digite 8 números (Ex: 12345000 ou 12345-000). Deixe em branco se não quiser informar.")

# Coleta do SINTOMA ou COMPORTAMENTO
sintoma = input(f"\n🤒 Descreva o SINTOMA ou COMPORTAMENTO de {nome_pet} sobre o qual quer saber mais: ")

# Resume os dados coletados antes de processar ---
print("\n--- 📝 RESUMO DOS DADOS ---")
print(f"- NOME: {nome_pet}")
print(f"- ESPÉCIE: {especie.capitalize()}")
print(f"- IDADE: {idade.capitalize()}")
print(f"- RAÇA: {raca if raca else 'Sem raça definida'}")
print(f"- PORTE: {porte.capitalize() if porte != 'não aplicável' else 'Não aplicável para a espécie'}")
print(f"- SEXO: {sexo.capitalize()}")
print(f"- CASTRADO? {castrado.capitalize()}")
print(f"- CEP: {cep}")
print(f"- SINTOMA/COMPORTAMENTO: {sintoma}")
print("---------------------------------------")

# Executa oos Agentes
if not sintoma:
    print("\n❌ Você esqueceu de digitar o sintoma! O VetBot precisa saber disso para buscar informações.")
else:
    print("\n--- 🔍 Agente 1 (Buscador) em ação... Buscando informações confiáveis... ---\n")
    informacoes_sintoma = vetbot_buscador(sintoma, especie, idade, raca, porte, sexo, castrado, cep, data_de_hoje)
    print("\n--- ✅ Agente 1 (Buscador) Concluído. Resultados brutos encontrados. ---\n")
    print("--------------------------------------------------------------")

    print("\n--- 🧠 Agente 2 (Filtrador) em ação... Analisando e filtrando informações... ---\n")
    informacoes_embasadas = vetbot_filtrador(sintoma, especie, idade, raca, porte, sexo, castrado, cep, informacoes_sintoma)
    print("\n--- ✅ Agente 2 (Filtrador) Concluído. Informações filtradas e sintetizadas. ---\n")
    print("--------------------------------------------------------------")

    print("\n--- 🩺 Agente 3 (Revisor Veterinário) em ação... Elaborando a resposta final segura... ---\n")
    resposta_final = vetbot_revisor(nome_pet, sintoma, especie, idade, raca, porte, sexo, castrado, cep, informacoes_embasadas)
    print("\n--- ✨ Resposta Final do VetBot para você e {}: ---\n".format(nome_pet))
    display(to_markdown(resposta_final))
    print("--------------------------------------------------------------")

🐾 Iniciando o VetBot – O Assistente Virtual do Seu Pet! 🐾
😺🐶 Qual é o NOME do seu pet? midori

🐾 Qual a ESPÉCIE do seu animal?
1. Cão 🐕
2. Gato 🐈
Digite o NÚMERO correspondente: 2
Você selecionou: Gato

🎂 Qual a FAIXA DE IDADE aproximada de Midori?
1. Filhote (até 6 meses)
2. Jovem (de 7 a 18 meses)
3. Adulto (de 19 meses a 7 anos)
4. Meia-idade (de 7 a 10 anos)
5. Idoso (acima de 10 anos)
6. Não sei / Outra
Digite o NÚMERO correspondente: 4
Você selecionou: Meia-idade (de 7 a 10 anos)

🦴 Qual a RAÇA de Midori? (OPCIONAL. Ex: Labrador, Siamês) vira lata

📏 Porte não se aplica para gatos.

🤔 Qual o SEXO de Midori (gato)?
1. Macho
2. Fêmea
3. Não sei / Não informado
Digite o NÚMERO correspondente: 2
Você selecionou: Fêmea

❤️ Midori é CASTRADO(A)?
1. Sim
2. Não
3. Não sei
Digite o NÚMERO correspondente: 1
Você selecionou: Sim

📍 Para sugerir hospitais/clínicas próximos, qual o CEP da sua região? (OPCIONAL) 05024000
CEP informado: 05024000

🤒 Descreva o SINTOMA ou COMPORTAMENTO de Midori 

> Olá! 👋 Entendo sua preocupação com os vômitos da Midori. Como veterinário, sei que vômitos em gatos podem ter diversas causas, e é importante investigar para garantir o bem-estar dela. No entanto, **é fundamental lembrar que este é apenas um auxílio informativo e não substitui uma consulta veterinária presencial com um profissional qualificado.**
> 
> **Nível de Urgência Aparente:** Consulta recomendada.
> 
> **O que pode estar acontecendo:**
> 
> Gatos como a Midori, de meia-idade (7-10 anos), podem apresentar vômitos por vários motivos. As causas mais comuns incluem:
> 
> *   **Bolas de pelo:** Gatos se limpam muito e acabam engolindo pelos, que podem se acumular no estômago.
> *   **Alimentação:** A causa pode ser tanto a ingestão de algo diferente (como lixo ou plantas) quanto alergias ou intolerâncias a algum ingrediente da ração. Evite oferecer alimentos não recomendados para gatos ou restos de comida.
> *   **Outras causas:** Várias outras condições podem levar a vômitos.
> 
> **O que observar em casa:**
> 
> É importante estar atento a algumas características dos vômitos e do comportamento da Midori, pois isso pode ajudar o veterinário a entender o que está acontecendo:
> 
> *   Com que frequência ela vomita?
> *   Qual a cor e o conteúdo do vômito?
> *   O vômito acontece logo após as refeições, ou em outros momentos?
> *   Ela tem outros sintomas, como diarreia, falta de apetite ou perda de peso?
> 
> **O que não fazer:**
> 
> *   Não medique a Midori por conta própria. Muitos medicamentos humanos são tóxicos para gatos.
> *   Não force a alimentação se ela estiver sem apetite.
> *   Evite dietas caseiras sem orientação profissional, pois elas podem não fornecer todos os nutrientes que ela precisa.
> 
> **Quando procurar um veterinário:**
> 
> Embora nem todo vômito seja motivo de pânico, alguns sinais indicam que é preciso procurar ajuda veterinária:
> 
> *   Se houver sangue no vômito
> *   Se ela estiver tentando vomitar, mas não consegue
> *   Se a barriga dela estiver inchada
> *   Se você suspeitar que ela comeu algo tóxico
> *   Se ela tiver febre ou estiver muito quieta e abatida
> *   Se as gengivas dela estiverem pálidas ou amareladas
> *   Se ela tiver diarreia
> 
> **O que o veterinário poderá fazer:**
> 
> O veterinário fará um exame físico completo da Midori e poderá solicitar alguns exames para identificar a causa do vômito.
> 
> **Enquanto aguarda a consulta:**
> 
> *   Mantenha a Midori hidratada, oferecendo água fresca com frequência.
> *   Observe atentamente o comportamento dela e anote as características dos vômitos.
> 
> **Hospitais e Clínicas Veterinárias 24 Horas Próximos:**
> 
> Para atendimento veterinário urgente, você pode contatar os seguintes locais:
> 
> *   **Hospital Veterinário Pompéia:** Av. Pompéia, 699 - São Paulo, SP. Tel: (11) 3673-9455 / (11) 3164-0182
> *   **Pet Care – Centro Veterinário 24 horas:** Av. Pacaembu, 1839 – São Paulo, SP. Tel: (11) 4305-9755
> *   **Koala Hospital Animal:** Av. Brigadeiro Luís Antônio, 4869 – São Paulo, SP. Tel: (11) 3845-3244
> *   **Hospital Veterinário Rebouças:** Av. Rebouças, 861 – São Paulo, SP. Tel: (11) 3062-3011
> *   **Veterinário Popular:** Rua Clélia, 731 - Barra Funda, São Paulo, SP.
> *   **Only Pet:** Avenida Brasil, 582 - Vila Correa, São Paulo, SP. Tel.: (11) 4677-5145 / (11) 2359-6664
> 
> *Observação:* Confirme se os locais listados acima ainda prestam serviços 24 horas.
> 
> Lembre-se que o vômito em gatos nunca deve ser ignorado, especialmente se for frequente ou acompanhado de outros sintomas. Não hesite em procurar um veterinário para garantir a saúde e o bem-estar da Midori.
> 
> **Reforço que sou apenas um assistente virtual e não posso substituir a avaliação e o atendimento de um médico veterinário.**
> 


--------------------------------------------------------------
